#Imports


In [0]:

# IMPORTS REFERENTES AS BIBLIOTECAS USADAS NO PROJETO

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
import gc
import time
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing
from scipy.stats import friedmanchisquare
!pip install -q SwarmPackagepy
!git clone https://github.com/IgorSouza21/SecundaryStructurePrediction.git

Using TensorFlow backend.


Cloning into 'SecundaryStructurePrediction'...
remote: Enumerating objects: 525, done.
remote: Counting objects: 100% (525/525), done.
remote: Compressing objects: 100% (522/522), done.
remote: Total 525 (delta 3), reused 521 (delta 1), pack-reused 0
Receiving objects: 100% (525/525), 1.88 MiB | 3.44 MiB/s, done.
Resolving deltas: 100% (3/3), done.


#Dataset

In [0]:
# ETAPA REFERENTE AO PRE PROCESSAMENTO E CONSTRUÇÃO DA BASE

import glob
import pickle
import pandas as pd
import keras

def conversion8to3(value):
    #     helix (G, H and I), strand (E and B) and loop (S, T, and _)
    c = {"H": ["G", "H", "I"], "S": ["E", "B"], "_": ["S", "T", "_", "?"]}
    for t in c.items():
        if value in t[1]:
            return t[0]


def qualquercoisa(v):
    if v == "?":
        return "_"
    else:
        return v
        

def loadfolderproteins(path):
    base = pd.DataFrame()
    res = []
    dssp8 = []
    dssp3 = []
    for i in glob.glob(path):
        f = open(i, "r")
        x = f.readlines()
        f.close()
        prot = x[0].split("RES:")[1]
        prot = prot.split(",")
        prot.pop()
        res.append("".join(prot))
        lb = x[1].split("DSSP:")[1]
        lb = lb.split(",")
        lb.pop()
        lba = [qualquercoisa(bl) for bl in lb]
        lb3 = [conversion8to3(bl) for bl in lb]
        dssp8.append("".join(lba))
        dssp3.append("".join(lb3))

    base["RES"] = res
    base['DSSP8'] = dssp8
    base['DSSP3'] = dssp3
    base.to_csv("SecundaryStructurePrediction/NewCB513.csv", index=False)


def load(X, size, window, classe="DSSP3"):
    data = X['RES']
    classes = X[classe]
    positions = {'A':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'K':9,'L':10,'M':11,'N':12,
                 'P':13,'Q':14,'R':15,'S':16,'T':17,'V':18,'W':19,'Y':20, 'X': 21, 'B': 22, 'Z': 23}
    classes_conv = {'S': 0, 'I': 1, 'G': 2, 'E': 3, 'T': 4, 'H': 5, 'B': 6, '_': 7}
    res = []
    for i in range(len(data)):
        for j in range(len(data[i])):
            res.append(int(positions[data[i][j]]))
    for i in range(size):
        res.insert(0, 0)
        res.append(0)
    data =[]
    for i in range(len(X['RES'])):
        data.append(res[i:window + i])
#         data[i].append(classes[i])
        data[i].append(classes_conv[classes[i]])
    columns = [[]]*(window + 1)
    for i in range(window):
        columns[i] = "Element" + str(i)
    columns[window] = 'Class'
    data = pd.DataFrame(data, columns = columns)
    
    return data


def load_data(window):
    size = int(window/2)
    proteins = pd.read_csv('SecundaryStructurePrediction/NewCB513.csv')
    res = load(proteins.iloc[0], size, window)
    for i in range(1,len(proteins)):
        aux = load(proteins.iloc[i], size, window,classe='DSSP3')
        res = res.append(aux,ignore_index = True)
        
    X = res.drop(['Class'], axis=1)
    y = res['Class']
    y = np.array(y)
    for x in range(len(y)):
        if y[x] == 5:
            y[x] = 1
        elif y[x] == 7:
            y[x] = 2
        
    return X,y#pd.DataFrame(keras.utils.to_categorical(y))

loadfolderproteins("SecundaryStructurePrediction/513_distribute/*.all")
#dataset, labels = load_data(15)

      
#print(labels)


#Functions

In [0]:
# FUNÇÕES UTILIZADAS NO PROJETO 

def amostra_estrat(tam , df , classe):
    classes = df[classe].unique()
    qtde_por_classe = round(tam / len(classes))
    amostras_por_classe = []
    for c in classes:
        indices_c = df[classe] == c
        obs_c = df[indices_c]
        amostra_c = obs_c.sample(qtde_por_classe)
        amostras_por_classe.append(amostra_c)
    amostra_estratificada = pd.concat(amostras_por_classe)
    return amostra_estratificada
  
windowsize = 5
arrayfitnessGWO = []

model = Sequential() # Instanciando o modelo
model.add(Dense(120, input_dim = windowsize, activation="relu",name='dense1'))
model.add(Dropout(0.3))
model.add(Dense(60,activation='relu',name='dense2'))
model.add(Dense(3, activation="softmax",name='dense3')) # Instanciando o modelo
model.compile( loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy'])

def normalize(df1):
    x = df1.values.astype(float)
    min_max_scaler = preprocessing.MinMaxScaler()
    scaled = min_max_scaler.fit_transform(x)
    df_normalized = pd.DataFrame(scaled)
    return df_normalized 
  
def creatpopuMSRA(sizepopu,sizecromo):    
    limit = np.sqrt(6/float(sizecromo))
    chromossomos = [np.random.uniform(low = -limit,high = limit, size = sizecromo) for x in range(sizepopu)]
    return chromossomos

def fitnessGWO(chromossome,Xtrain,ytrain,Xval,yval):
    
    w1 = np.reshape(np.split(np.array(chromossome[0:600]),120),(5,120))           #np.split(np.array(chromossome[0:len(chromossome)-6]), 50)
    w2 = np.reshape(np.split(np.array(chromossome[600:7800]),60),(120,60))           #np.reshape(chromossome[len(chromossome)-7:len(chromossome)-1], (6, 1))
    w3 = np.reshape(chromossome[7800:7980], (60,3))
    
    model.get_layer('dense1').set_weights((np.array(w1), np.zeros(120)))
    model.get_layer('dense2').set_weights((np.array(w2), np.zeros(60)))
    model.get_layer('dense3').set_weights((np.array(w3), np.zeros(3)))
    model.fit(Xtrain, ytrain, epochs=100, batch_size=512, verbose = False)
    retorno = model.evaluate(Xval,yval, verbose = False)

    if len(arrayfitnessGWO) == 0:
        model_json = model.to_json()
        with open("modelGWO.json", "w") as json_file:
            json_file.write(model_json)
        model.save_weights("modelGWO.h5")

    atual = retorno[1]

    if len(arrayfitnessGWO) > 1:
        if atual > arrayfitnessGWO[np.argmax(arrayfitnessGWO)]:
            model_json = model.to_json()
            with open("modelGWO.json", "w") as json_file:
                json_file.write(model_json)
            model.save_weights("modelGWO.h5")

    arrayfitnessGWO.append(atual)

    return atual

def loadmodelGWO():
    json_file = open('modelGWO.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("modelGWO.h5")
    loaded_model.compile( loss='binary_crossentropy', optimizer="adam", 
              metrics=['accuracy'])

    return loaded_model


from SwarmPackagePy.intelligence import sw


class gwo(sw):

    def __init__(self, n, function, lb, ub, dimension, iteration,Xtrain,ytrain,Xval,yval,init = None):


        super(gwo, self).__init__()

        if init is None:
            self.__agents = np.random.uniform(lb, ub, (n, dimension))
        else:
            self.__agents = np.array(init)
            
        self._points(self.__agents)
        alpha, beta, delta = self.__get_abd(n, function,Xtrain,ytrain,Xval,yval)

        Gbest = alpha

        for t in range(iteration):

            a = 2 - 2 * t / iteration

            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            A1 = 2 * r1 * a - a
            C1 = 2 * r2

            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            A2 = 2 * r1 * a - a
            C2 = 2 * r2

            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            A3 = 2 * r1 * a - a
            C3 = 2 * r2

            Dalpha = abs(C1 * alpha - self.__agents)
            Dbeta = abs(C2 * beta - self.__agents)
            Ddelta = abs(C3 * delta - self.__agents)

            X1 = alpha - A1 * Dalpha
            X2 = beta - A2 * Dbeta
            X3 = delta - A3 * Ddelta
            self.__agents = (X1 + X2 + X3) / 3

            self.__agents = np.clip(self.__agents, lb, ub)
            self._points(self.__agents)

            alpha, beta, delta = self.__get_abd(n, function,Xtrain,ytrain,Xval,yval)
            if function(alpha,Xtrain,ytrain,Xval,yval) > function(Gbest,Xtrain,ytrain,Xval,yval):
                Gbest = alpha

        self._set_Gbest(Gbest)
        alpha, beta, delta = self.__get_abd(n, function,Xtrain,ytrain,Xval,yval)
        self.__leaders = list(alpha), list(beta), list(delta)

    def __get_abd(self, n, function,Xtrain,ytrain,Xval,yval):

        result = []
        fitness = [(function(self.__agents[i],Xtrain,ytrain,Xval,yval), i) for i in range(n)]
        fitness.sort(reverse=True)

        for i in range(3):
            result.append(self.__agents[fitness[i][1]])

        return result

    def get_leaders(self):

        return list(self.__leaders)

#Main

In [0]:
# MAIN DO PROJETO ONDE É REALIZADO OS TESTES

ite = 2    # Quantidade de gerações dos algoritmos evolutivos 
iteKfold = 1 # Quantidade de iterações que vai ser executado o cross validation de k = 10
scoresfinalMLP = [] # Array para salvar os score de validação da MLP
cont = 0 # Seed para o shuffle da base de dados
popu = 3 # Tamanho da população
kvalue = 10

dataset,labels = load_data(windowsize) # CARREGANDO A BASE
dataset = normalize(dataset) # NORMALIZANDO A BASE
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=False,save_best_only=True) # UTILIZANDO CHECKPOINTER PRA SALVAR CONFIGURAÇÃO DA MLP DURANTE AS EPOCAS

testGWO = []
testMLP = []

k = StratifiedKFold(kvalue,True,1)
# k = KFold(kvalue,True,1) # Instancia da classe Kfold com k = 10

for x in range(iteKfold): # For da quantidade de vezes que ira ser executado o cross validation de k = 10
    
    c = k.split(dataset,labels)
    labels = keras.utils.to_categorical(labels)
    for train_index,test_index in c: # For do Kfold com K = 10

        X_train, X_test = np.array(dataset)[train_index], np.array(dataset)[test_index] # Separando o conjunto de Treino separado a cima em treino e validação
        y_train, y_test = np.array(labels)[train_index], np.array(labels)[test_index] # Separando o conjunto de Treino separado a cima em treino e validação

        X_trainK, X_valK, y_trainK, y_valK = train_test_split(X_train, y_train, test_size=0.2)


        ############ CREATE POPU ######

        inicialpopulation = np.array(creatpopuMSRA(popu,7980)) # Criando a população inicial
        sizepop = inicialpopulation.shape[0] # Tamanho da população
        dimpop = inicialpopulation.shape[1] # Dimensão da população

        ############ CREATE POPU #########

        ######## MLP ############

        modelMLP = Sequential() # Instanciando o modelo
        modelMLP.add(Dense(120, input_dim = windowsize, activation="relu"))
        modelMLP.add(Dropout(0.3))
        modelMLP.add(Dense(60,activation='relu'))
        modelMLP.add(Dense(3, activation="softmax")) # Instanciando o modelo
        modelMLP.compile( loss='categorical_crossentropy', optimizer="adam",metrics=['accuracy']) # Instanciando o modelo

        modelMLP.fit(X_trainK, y_trainK,callbacks = [checkpointer], epochs=100, batch_size=516,validation_data=(X_valK,y_valK),verbose=False) 

        ################ GWO ############################

        grey = gwo(sizepop,fitnessGWO,-1,1,dimpop,ite,X_trainK,y_trainK,X_valK,y_valK,init=inicialpopulation.copy(),)# Rodando o GWO com os conjuntos de validação e treinamento e com o númer

        modelGWO = loadmodelGWO() # Carregando o melhor modelo gerado pelo GWO para teste

        testGWO.append(modelGWO.evaluate(X_test,y_test, verbose = False)[1]) # Avaliando o melhor modelo do GWO com o conjunto de teste
        modelMLP.load_weights('model.weights.best.hdf5')
        testMLP.append(modelMLP.evaluate(X_test,y_test, verbose = False)[1])
        arrayfitnessGWO = []
        
        print(testMLP)
        print(testGWO)


    dataset, labels = shuffle(dataset, labels, random_state=cont) # Depois que termina as 10 iterações de um Kfold dou shuffle
    cont += 1 # Valor da seed do shufle sendo somado +1

datacsv = []

for ite2 in range(iteKfold*kvalue):
    aux = []
    aux.append(testMLP[ite2])
    aux.append(testGWO[ite2])
    datacsv.append(aux)    


datacsv = pd.DataFrame(datacsv, columns=['MLP','GWO'])

dataplot = datacsv.boxplot(column=['MLP','GWO'],figsize=(12,8))
plt.xlabel('Algorithms')
plt.ylabel('Acuracy score')
plt.title('Result')   

print("media")
print(np.mean(testMLP))
print(np.mean(testGWO))

print("mediana")
print(np.median(testMLP))
print(np.median(testGWO))

print("std")
print(np.std(testMLP))
print(np.std(testGWO))


